# Base Model using all Train Data

In [1]:
import pandas as pd
import numpy as np
import os 

import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Private DataSet path: ../input/kddbr2018dataset/kddbr-2018-dataset/dataset. This dataset is the same of competitions
#
path = '../input/'
print(os.listdir(path))

['field-22.csv', 'field-20.csv', 'sample-submission.csv', 'field-3.csv', 'soil_data.csv', 'field-18.csv', 'field-23.csv', 'field-17.csv', 'field-2.csv', 'field-27.csv', 'field-24.csv', 'field-10.csv', 'field-11.csv', 'field-26.csv', 'field-21.csv', 'test.csv', 'field-25.csv', 'field-19.csv', 'field-7.csv', 'field-15.csv', 'field-8.csv', 'field-12.csv', 'field-0.csv', 'field-1.csv', 'field-14.csv', 'train.csv', 'field-5.csv', 'field-16.csv', 'field-13.csv', 'field-9.csv', 'field-6.csv', 'field-4.csv']


## Train and test datasets

Basic data containing palm tree information

In [3]:
df_train = pd.read_csv(os.path.join(path, 'train.csv'))
df_test  = pd.read_csv(os.path.join(path, 'test.csv'))
df_all = pd.concat([df_train, df_test])

print(df_train.shape, df_test.shape, df_all.shape)

(5243, 7) (4110, 6) (9353, 7)


In [4]:
def to_date(df):
    return pd.to_datetime((df.harvest_year*10000+df.harvest_month*100+1)\
                                  .apply(str),format='%Y%m%d')
# Add date variable 
for d in [df_train, df_test, df_all]:
    d['date'] = to_date(d)

## Field data (field_*.csv)

These files hold atmospheric data from January 2002 to December 2017, and can be used to estimate the weather conditions during the development of the plant. Notice that weather does influence the production. Using only a single month prior to harvest is probably too little data. Participants should decide how far back in the past they want to look when training models.



In [5]:
# read
df_field = pd.read_csv(path+'field-0.csv')
df_field['field'] = 0
for i in range(1, 28):
    _df_field = pd.read_csv(path+'field-{}.csv'.format(i))
    _df_field['field'] = i
    df_field = pd.concat([df_field, _df_field])

# remove duplicates
df_field = df_field.drop_duplicates()

# Group 
df_field = df_field.groupby(['month', 'year', 'field']).mean().reset_index()
print(df_field.shape)
df_field.head()

(5376, 11)


,month,year,field,temperature,dewpoint,windspeed,Soilwater_L1,Soilwater_L2,Soilwater_L3,Soilwater_L4,Precipitation
0,1,2002,0,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
1,1,2002,1,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
2,1,2002,2,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
3,1,2002,3,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55
4,1,2002,4,26.008,24.434,1.8453,0.32984,0.32597,0.31477,0.29513,361.55


In [6]:
# df_all
df_all   = pd.merge(df_all, df_field, left_on=['harvest_year', 'harvest_month','field'], 
                    right_on=['year', 'month', 'field'], how='inner').reset_index()

print(df_all.shape)
df_all.head()

(9353, 19)


,index,Id,age,field,harvest_month,harvest_year,production,type,date,month,year,temperature,dewpoint,windspeed,Soilwater_L1,Soilwater_L2,Soilwater_L3,Soilwater_L4,Precipitation
0,0,0,19,0,1,2004,0.064071,5,2004-01-01,1,2004,26.132,24.661,1.8766,0.35274,0.35192,0.34844,0.33385,360.91
1,1,1532,13,0,1,2004,0.283228,2,2004-01-01,1,2004,26.132,24.661,1.8766,0.35274,0.35192,0.34844,0.33385,360.91
2,2,4204,4,0,1,2004,0.106263,5,2004-01-01,1,2004,26.132,24.661,1.8766,0.35274,0.35192,0.34844,0.33385,360.91
3,3,1,19,0,2,2004,0.047658,5,2004-02-01,2,2004,25.295,24.401,1.9206,0.36361,0.36376,0.36411,0.36357,484.67
4,4,1533,13,0,2,2004,0.182068,2,2004-02-01,2,2004,25.295,24.401,1.9206,0.36361,0.36376,0.36411,0.36357,484.67


In [7]:
df_all.columns

Index(['index', 'Id', 'age', 'field', 'harvest_month', 'harvest_year',
       'production', 'type', 'date', 'month', 'year', 'temperature',
       'dewpoint', 'windspeed', 'Soilwater_L1', 'Soilwater_L2', 'Soilwater_L3',
       'Soilwater_L4', 'Precipitation'],
      dtype='object')

In [8]:
# Features i will duplicate with the past months
features  = ['temperature', 'dewpoint', 'windspeed', 'Precipitation', 'Soilwater_L1']

df_all    = df_all.drop(columns=['Soilwater_L2', 'Soilwater_L3','Soilwater_L4'])

In [9]:
df_group = df_all.groupby(['field', 'date']).mean().reset_index()[['field', 'date', 'production'] + features ]
df_group = df_group.sort_values(['field', 'date'])
print(df_group.shape)
df_group.head()

(4336, 8)


,field,date,production,temperature,dewpoint,windspeed,Precipitation,Soilwater_L1
0,0,2004-01-01,0.151187,26.132,24.661,1.8766,360.91,0.35274
1,0,2004-02-01,0.089973,25.295,24.401,1.9206,484.67,0.36361
2,0,2004-03-01,0.175152,25.610,24.651,1.9948,460.76,0.36399
3,0,2004-04-01,0.066224,26.328,24.753,1.9063,350.02,0.35677
4,0,2004-05-01,0.214931,26.566,24.940,1.8569,186.72,0.36002


In [10]:
# Collect shift values of variables in all features time
period = 2

new_features = {}
for f in features:
    new_features[f] = []
    for i in range(1, period):
        new_features[f].append('{}_{}'.format(f, i))
        df_group['{}_{}'.format(f, i)] = df_group[f].shift(i).fillna(df_group[f].mean())
        #df_group['{}_{}'.format(f, i)] = df_group[f].rolling(i, min_periods=1).mean().fillna(df_group.temperature.mean())

In [11]:
df_group= df_group.drop(features+['production'], axis=1)
df_group.head()

,field,date,temperature_1,dewpoint_1,windspeed_1,Precipitation_1,Soilwater_L1_1
0,0,2004-01-01,27.418226,23.813569,2.13645,248.002634,0.309693
1,0,2004-02-01,26.132000,24.661000,1.87660,360.910000,0.352740
2,0,2004-03-01,25.295000,24.401000,1.92060,484.670000,0.363610
3,0,2004-04-01,25.610000,24.651000,1.99480,460.760000,0.363990
4,0,2004-05-01,26.328000,24.753000,1.90630,350.020000,0.356770


In [12]:
df_all = df_all.drop(['index', 'month', 'year'], axis=1)
df_all = pd.merge(df_all, df_group, left_on=['field', 'date'], right_on=['field','date'], how='inner').reset_index()

print(df_all.shape)
df_all.head()

(9353, 19)


,index,Id,age,field,harvest_month,harvest_year,production,type,date,temperature,dewpoint,windspeed,Soilwater_L1,Precipitation,temperature_1,dewpoint_1,windspeed_1,Precipitation_1,Soilwater_L1_1
0,0,0,19,0,1,2004,0.064071,5,2004-01-01,26.132,24.661,1.8766,0.35274,360.91,27.418226,23.813569,2.13645,248.002634,0.309693
1,1,1532,13,0,1,2004,0.283228,2,2004-01-01,26.132,24.661,1.8766,0.35274,360.91,27.418226,23.813569,2.13645,248.002634,0.309693
2,2,4204,4,0,1,2004,0.106263,5,2004-01-01,26.132,24.661,1.8766,0.35274,360.91,27.418226,23.813569,2.13645,248.002634,0.309693
3,3,1,19,0,2,2004,0.047658,5,2004-02-01,25.295,24.401,1.9206,0.36361,484.67,26.132000,24.661000,1.87660,360.910000,0.352740
4,4,1533,13,0,2,2004,0.182068,2,2004-02-01,25.295,24.401,1.9206,0.36361,484.67,26.132000,24.661000,1.87660,360.910000,0.352740


### soil_data.csv

Information on the soil on which each field is

In [13]:
df_soil = pd.read_csv(path+'soil_data.csv')
print(df_soil.shape)
df_soil.head()

(28, 73)


,field,BDRICM_BDRICM_M,BDRLOG_BDRLOG_M,BDTICM_BDTICM_M,BLDFIE_sl1,BLDFIE_sl2,BLDFIE_sl3,BLDFIE_sl4,BLDFIE_sl5,BLDFIE_sl6,...,SLTPPT_sl5,SLTPPT_sl6,SLTPPT_sl7,SNDPPT_sl1,SNDPPT_sl2,SNDPPT_sl3,SNDPPT_sl4,SNDPPT_sl5,SNDPPT_sl6,SNDPPT_sl7
0,4,200,7,6973,1345,1308,1361,1413,1486,1503,...,21,19,20,47,48,47,42,40,40,39
1,3,200,9,7272,1297,1287,1323,1428,1492,1508,...,23,22,22,44,45,43,40,36,37,36
2,2,200,7,7281,1266,1249,1310,1387,1463,1491,...,21,21,22,46,46,45,40,39,39,39
3,1,200,6,7457,1297,1277,1345,1409,1480,1506,...,21,21,21,46,47,46,42,40,40,40
4,7,200,8,6771,1305,1289,1333,1438,1497,1510,...,22,21,22,44,45,44,40,38,38,37


In [14]:
# Join datasets
df_all_soil = pd.merge(df_all, df_soil, on='field', how='inner')
print(df_all_soil.shape)
df_all_soil.head()

(9353, 91)


,index,Id,age,field,harvest_month,harvest_year,production,type,date,temperature,...,SLTPPT_sl5,SLTPPT_sl6,SLTPPT_sl7,SNDPPT_sl1,SNDPPT_sl2,SNDPPT_sl3,SNDPPT_sl4,SNDPPT_sl5,SNDPPT_sl6,SNDPPT_sl7
0,0,0,19,0,1,2004,0.064071,5,2004-01-01,26.132,...,22,22,23,44,45,44,39,38,37,36
1,1,1532,13,0,1,2004,0.283228,2,2004-01-01,26.132,...,22,22,23,44,45,44,39,38,37,36
2,2,4204,4,0,1,2004,0.106263,5,2004-01-01,26.132,...,22,22,23,44,45,44,39,38,37,36
3,3,1,19,0,2,2004,0.047658,5,2004-02-01,25.295,...,22,22,23,44,45,44,39,38,37,36
4,4,1533,13,0,2,2004,0.182068,2,2004-02-01,25.295,...,22,22,23,44,45,44,39,38,37,36


### Feature Importance Measures

Find the main features for the production target. Uses a RandomRorest to identify features


In [15]:
df_all.columns

Index(['index', 'Id', 'age', 'field', 'harvest_month', 'harvest_year',
       'production', 'type', 'date', 'temperature', 'dewpoint', 'windspeed',
       'Soilwater_L1', 'Precipitation', 'temperature_1', 'dewpoint_1',
       'windspeed_1', 'Precipitation_1', 'Soilwater_L1_1'],
      dtype='object')

In [16]:
## Import the random forest model.
from sklearn.ensemble import RandomForestRegressor

## This line instantiates the model. 
rf = RandomForestRegressor() 

# data
df      = df_all_soil[~df_all_soil.production.isna()]
X_train = df.drop(['production', 'date', 'Id', 'index'], axis=1)
y_train = df.production.values

## Fit the model on your training data.
rf.fit(X_train, y_train) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [17]:
# feature_importances
feature_importances = pd.DataFrame(rf.feature_importances_, 
                                   index = X_train.columns, 
                                   columns=['importance']).sort_values('importance', ascending=False).reset_index()
feature_importances.head()

,index,importance
0,Soilwater_L1,0.186999
1,age,0.154525
2,Precipitation_1,0.071036
3,Precipitation,0.058082
4,type,0.053482


## Base Model 

Creation of a baseline model to finalize the competition submission pipeline. The idea is to create the most basic for future improvements.

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import sklearn.model_selection

### Prepare Dataset

In [19]:
# Load Dataset
# y~X
df_train = df_all_soil[~df_all_soil.production.isna()]
X        = df_train.drop(['production', 'date', 'Id'], axis=1)

#Filter importance
features = list(feature_importances['index'].values)[:15]
X        = X[features]
# y
y        = df.production.values

# normalize
scaler = StandardScaler()
norm_X = scaler.fit_transform(X)

# Split
# X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(norm_X, y, test_size=0.2, random_state=1)
# (X_train.shape, X_test.shape)

### Train Model

In [20]:
base_model = RandomForestRegressor()
base_model.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

### Submission

It makes the predict of the basic model and creates the sample to substrate in kaggle, finishing the complete pipeline

In [21]:
df_test = df_all[df_all.production.isna()]
print(df_test.shape)
df_test.tail(2)

(4110, 19)


,index,Id,age,field,harvest_month,harvest_year,production,type,date,temperature,dewpoint,windspeed,Soilwater_L1,Precipitation,temperature_1,dewpoint_1,windspeed_1,Precipitation_1,Soilwater_L1_1
9351,9351,9097,8,3,9,2017,NaN,5,2017-09-01,28.604,23.572,2.3288,0.27126,92.301,28.502,23.580,2.0800,151.720,0.27522
9352,9352,9098,8,3,10,2017,NaN,5,2017-10-01,28.660,23.361,2.6919,0.25575,196.660,28.604,23.572,2.3288,92.301,0.27126


Build test dataset to predict 

In [22]:
#Filter importance
X  = df_test[features]
X  = scaler.transform(X) # normalize

# y
y = df.production.values

In [23]:
prod = base_model.predict(X) 
prod[:10]

array([0.16365907, 0.16365907, 0.16365907, 0.16365907, 0.16365907,
       0.15658192, 0.15658192, 0.15658192, 0.15658192, 0.15658192])

In [24]:
## create a submission.csv
import math
f = open('../submissions/submission_all_data.csv', 'w')
f.write("Id,production\n")
for i in range(len(df_test.Id.values)):
    _id = df_test.Id.values[i]
    p   = math.fabs(prod[i])
    f.write("{},{}\n".format(_id, p))
f.close()